In [1]:
import uuid
print(':'.join(['{:02x}'.format((uuid.getnode() >> ele) & 0xff) for ele in range(0, 8*6, 8)][::-1]))

89:cb:c4:10:87:7c


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, filedialog
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
from PIL import Image, ImageTk
import numpy as np
import threading
import queue
import io
import time
import secrets
import uuid
from cryptography.fernet import Fernet
import os
import getpass
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend to avoid conflicts with Tkinter
import matplotlib.pyplot as plt

# --- Image Processing ---
def image_to_bits(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((16, 16))
    pixel_array = np.array(img)
    binary_pixels = np.unpackbits(pixel_array).tolist()
    return binary_pixels

def bits_to_image(bits, output_path, original_size=(16, 16)):
    byte_array = np.packbits(bits)
    img_array = byte_array.reshape(original_size)
    img = Image.fromarray(img_array.astype(np.uint8))
    img.save(output_path, format='PNG')
    return output_path

# --- QTRNG Key Generation ---
def generate_qtrng_key(total_length, num_qubits=24, log_queue=None):
    backend = BasicSimulator()
    key = []
    rounds = (total_length + num_qubits - 1) // num_qubits
    
    for i in range(rounds):
        qr = QuantumRegister(num_qubits, 'q')
        cr = ClassicalRegister(num_qubits, 'c')
        ckt = QuantumCircuit(qr, cr)
        
        for j in range(num_qubits):
            ckt.h(j)
        ckt.measure(qr, cr)
        
        result = backend.run(transpile(ckt, backend), shots=1).result()
        counts = result.get_counts()
        bitstring = list(counts.keys())[0]
        round_key = [int(bit) for bit in bitstring.replace(' ', '')]
        
        if log_queue:
            log_queue.put(f"Generating Quantum Key for Image State Transmission, Round {i+1}: {len(round_key)} bits")
        print(f"Debug: Generating Quantum Key for Image State Transmission, Round {i+1}: Key = {round_key[:10]}..., length = {len(round_key)}")
        
        key.extend(round_key)
    
    key = key[:total_length]
    if log_queue:
        log_queue.put(f"Quantum Key Generated: {len(key)} bits")
    print(f"Debug: Final Quantum Key = {key[:20]}..., length = {len(key)}")
    return key

# --- NEQR Representation (Quantum Circuit) ---
def bits_to_neqr(bits, log_queue=None):
    num_pixels = 256  # 16x16 image
    position_qubits = 8  # 2^8 = 256 positions
    color_qubits = 8  # 8 bits for grayscale
    
    qr_pos = QuantumRegister(position_qubits, 'pos')
    qr_color = QuantumRegister(color_qubits, 'color')
    cr = ClassicalRegister(position_qubits + color_qubits, 'c')
    qc = QuantumCircuit(qr_pos, qr_color, cr)
    
    # Step 1: Initialize position qubits in superposition
    for i in range(position_qubits):
        qc.h(qr_pos[i])
    
    # Step 2: Encode color information for each pixel
    pixels = [bits[i:i+8] for i in range(0, len(bits), 8)]
    for pixel_idx in range(num_pixels):
        y = pixel_idx // 16
        x = pixel_idx % 16
        pos_binary = format(pixel_idx, '08b')
        color_binary = pixels[pixel_idx]
        
        # Apply X gates to set position qubits
        for i, bit in enumerate(pos_binary):
            if bit == '1':
                qc.x(qr_pos[i])
        
        # Apply controlled-X gates to set color qubits
        for i, bit in enumerate(color_binary):
            if bit == 1:
                qc.mcx(qr_pos, qr_color[i])
        
        # Reset position qubits for the next pixel
        for i, bit in enumerate(pos_binary):
            if bit == '1':
                qc.x(qr_pos[i])
    
    if log_queue:
        log_queue.put("NEQR quantum circuit created")
    print("Debug: NEQR circuit created")
    return qc, pixels

# --- Simulated Encryption (Confusion/Diffusion/Permutation/Scrambling) ---
def encrypt_simulated_image(pixels, log_queue=None):
    confusion_key = np.random.randint(0, 2, size=(256, 8)).tolist()
    confused_pixels = []
    for pixel, key in zip(pixels, confusion_key):
        confused_pixel = [b ^ k for b, k in zip(pixel, key)]
        confused_pixels.append(confused_pixel)
    
    diffused_pixels = confused_pixels.copy()
    for i in range(256):
        for j in range(8):
            if i > 0:
                diffused_pixels[i][j] ^= diffused_pixels[i-1][j]
    
    permutation = np.random.permutation(256)
    scrambled_pixels = [diffused_pixels[i] for i in permutation]
    
    if log_queue:
        log_queue.put("Simulated image encrypted (confusion/diffusion/permutation)")
    print("Debug: Simulated image encrypted")
    return scrambled_pixels, (confusion_key, permutation)

# --- Simulated Decryption ---
def decrypt_simulated_image(scrambled_pixels, encryption_keys, log_queue=None):
    confusion_key, permutation = encryption_keys
    
    inverse_permutation = np.argsort(permutation)
    diffused_pixels = [scrambled_pixels[i] for i in inverse_permutation]
    
    confused_pixels = diffused_pixels.copy()
    for i in range(255, -1, -1):
        for j in range(8):
            if i > 0:
                confused_pixels[i][j] ^= confused_pixels[i-1][j]
    
    pixels = []
    for pixel, key in zip(confused_pixels, confusion_key):
        pixel = [b ^ k for b, k in zip(pixel, key)]
        pixels.append(pixel)
    
    if log_queue:
        log_queue.put("Simulated image decrypted")
    print("Debug: Simulated image decrypted")
    return pixels

# --- Simulated Quantum Channel Transmission ---
def transmit_simulated_image(scrambled_pixels, log_queue=None):
    transmitted_bits = []
    for pixel in scrambled_pixels:
        transmitted_bits.extend(pixel)
    
    if log_queue:
        log_queue.put(f"Quantum Image State Transmission, sent {len(transmitted_bits)} bits")
    print(f"Debug: Transmitted bits = {transmitted_bits[:20]}..., length = {len(transmitted_bits)}")
    return transmitted_bits

# --- QKD Transmission ---
def transmit_qkd_image(bits, num_qubits=24, log_queue=None):
    start_time = time.time()
    
    qtrng_key = generate_qtrng_key(len(bits), num_qubits, log_queue)
    xored_bits = [b ^ k for b, k in zip(bits, qtrng_key)]
    
    neqr_circuit, pixels = bits_to_neqr(xored_bits, log_queue)
    
    scrambled_pixels, encryption_keys = encrypt_simulated_image(pixels, log_queue)
    
    transmitted_bits = transmit_simulated_image(scrambled_pixels, log_queue)
    
    print(f"QKD Image Transmission took {time.time() - start_time:.2f} seconds")
    return transmitted_bits, qtrng_key, encryption_keys, neqr_circuit

# --- Security Functions ---
def generate_otp():
    return ''.join([str(secrets.randbelow(10)) for _ in range(6)])

def verify_mac():
    allowed_mac = "89:cb:c4:10:87:7c"  # REPLACE WITH YOUR MAC ADDRESS HERE
    current_mac = ':'.join(['{:02x}'.format((uuid.getnode() >> ele) & 0xff) for ele in range(0, 8*6, 8)][::-1])
    print(f"Current MAC: {current_mac}")
    return current_mac == allowed_mac

def encrypt_and_store(data, usb_path, filename):
    fernet_key = Fernet.generate_key()
    cipher = Fernet(fernet_key)
    data_str = ''.join(map(str, data)) if isinstance(data, list) else str(data)
    encrypted_data = cipher.encrypt(data_str.encode())
    
    with open(os.path.join(usb_path, f"{filename}.bin"), 'wb') as f:
        f.write(encrypted_data)
    with open(os.path.join(usb_path, f"{filename}_fernet_key.bin"), 'wb') as f:
        f.write(fernet_key)
    
    return f"{filename}.bin", f"{filename}_fernet_key.bin"

# --- Login Window ---
class LoginWindow:
    def __init__(self, parent, callback, title):
        self.window = tk.Toplevel(parent)
        self.window.title(f"{title} Login")
        self.window.configure(bg="#1a1a1a")
        self.callback = callback
        self.otp = None

        self.style = ttk.Style()
        self.style.configure("TFrame", background="#1a1a1a")
        self.style.configure("TLabel", background="#1a1a1a", foreground="#00ffcc", font=("Orbitron", 12))
        self.style.configure("TEntry", fieldbackground="#333333", foreground="#00ffcc", font=("Orbitron", 12))
        self.style.configure("TButton", background="#00ffcc", foreground="#1a1a1a", font=("Orbitron", 12, "bold"))
        self.style.map("TButton", background=[("active", "#00cc99")])

        self.frame = ttk.Frame(self.window, padding="20")
        self.frame.grid(row=0, column=0, sticky="nsew")

        ttk.Label(self.frame, text="Login ID:").grid(row=0, column=0, padx=10, pady=10, sticky="w")
        self.id_entry = ttk.Entry(self.frame)
        self.id_entry.grid(row=0, column=1, padx=10, pady=10, sticky="ew")

        ttk.Label(self.frame, text="Password:").grid(row=1, column=0, padx=10, pady=10, sticky="w")
        self.pw_entry = ttk.Entry(self.frame, show="*")
        self.pw_entry.grid(row=1, column=1, padx=10, pady=10, sticky="ew")

        ttk.Label(self.frame, text="OTP:").grid(row=2, column=0, padx=10, pady=10, sticky="w")
        self.otp_entry = ttk.Entry(self.frame)
        self.otp_entry.grid(row=2, column=1, padx=10, pady=10, sticky="ew")

        ttk.Label(self.frame, text="USB Path:").grid(row=3, column=0, padx=10, pady=10, sticky="w")
        self.usb_entry = ttk.Entry(self.frame)
        self.usb_entry.insert(0, "E:\\")
        self.usb_entry.grid(row=3, column=1, padx=10, pady=10, sticky="ew")

        ttk.Button(self.frame, text="Generate OTP", command=self.generate_otp).grid(row=4, column=0, pady=20)
        ttk.Button(self.frame, text="Login", command=self.login).grid(row=4, column=1, pady=20)

    def generate_otp(self):
        self.otp = generate_otp()
        print(f"Generated OTP (check console): {self.otp}")

    def login(self):
        login_id = self.id_entry.get()
        password = self.pw_entry.get()
        otp = self.otp_entry.get()
        usb_path = self.usb_entry.get()

        if login_id != "admin" or password != "password123":
            messagebox.showerror("Error", "Invalid Login ID or Password")
            return
        if self.otp is None or otp != self.otp:
            messagebox.showerror("Error", "Invalid OTP")
            return
        if not verify_mac():
            messagebox.showerror("Error", "MAC Verification Failed")
            return
        if not os.path.exists(usb_path):
            messagebox.showerror("Error", "USB Path Not Found")
            return

        self.window.destroy()
        self.callback(usb_path)

# --- NEQR Circuit Display Window (Updated with Fixes) ---
class NEQRCircuitWindow:
    def __init__(self, parent, circuit):
        self.window = tk.Toplevel(parent)
        self.window.title("NEQR Quantum Circuit")
        self.window.configure(bg="#1a1a1a")
        self.window.geometry("1200x800")  # Large window for circuit display

        self.style = ttk.Style()
        self.style.configure("TFrame", background="#1a1a1a")
        self.style.configure("TLabel", background="#1a1a1a", foreground="#00ffcc", font=("Orbitron", 12))

        self.main_frame = ttk.Frame(self.window, style="TFrame")
        self.main_frame.pack(fill="both", expand=True, padx=20, pady=20)

        ttk.Label(self.main_frame, text="NEQR Quantum Circuit", font=("Orbitron", 16, "bold")).pack(pady=10)

        # Create a canvas with both horizontal and vertical scrollbars
        self.canvas_frame = ttk.Frame(self.main_frame)
        self.canvas_frame.pack(fill="both", expand=True)

        self.canvas = tk.Canvas(self.canvas_frame, bg="#333333")
        self.h_scrollbar = ttk.Scrollbar(self.canvas_frame, orient="horizontal", command=self.canvas.xview)
        self.v_scrollbar = ttk.Scrollbar(self.canvas_frame, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(xscrollcommand=self.h_scrollbar.set, yscrollcommand=self.v_scrollbar.set)

        self.h_scrollbar.pack(side="bottom", fill="x")
        self.v_scrollbar.pack(side="right", fill="y")
        self.canvas.pack(side="left", fill="both", expand=True)

        # Keep a reference to the PhotoImage to prevent garbage collection
        self.photo = None

        # Draw the circuit
        try:
            self.display_circuit(circuit)
        except Exception as e:
            print(f"Error displaying circuit: {str(e)}")
            ttk.Label(self.main_frame, text=f"Error displaying circuit: {str(e)}", foreground="red").pack(pady=10)

    def display_circuit(self, circuit):
        # Draw the circuit using Matplotlib
        try:
            # Use a smaller figure size to manage large circuits
            fig = plt.figure(figsize=(15, 10))
            circuit.draw('mpl', fold=25, ax=fig.gca())  # Use fold=25 to wrap the circuit for better visibility

            # Convert the Matplotlib figure to a PIL image
            buf = io.BytesIO()
            fig.savefig(buf, format="png", bbox_inches="tight", dpi=100)
            buf.seek(0)
            image = Image.open(buf)

            # Resize the image to fit within the window while maintaining aspect ratio
            max_width = 1000
            max_height = 600
            image.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)

            # Convert the PIL image to a Tkinter-compatible image
            self.photo = ImageTk.PhotoImage(image)

            # Update the canvas to fit the image
            self.canvas.create_image(0, 0, image=self.photo, anchor="nw")
            self.canvas.configure(scrollregion=(0, 0, image.width, image.height))

            # Close the Matplotlib figure to free memory
            plt.close(fig)
            print("Circuit successfully drawn and displayed.")
        except Exception as e:
            print(f"Error in display_circuit: {str(e)}")
            raise

# --- Sender GUI ---
class SenderGUI:
    def __init__(self, parent, channel_queue, status_queue):
        self.channel_queue = channel_queue
        self.status_queue = status_queue
        self.login_window = LoginWindow(parent, self.start_main_gui, "Sender")
        self.usb_path = None
        self.neqr_circuit = None

    def start_main_gui(self, usb_path):
        self.usb_path = usb_path
        self.window = tk.Toplevel(self.login_window.window.master)
        self.window.title("Quantum Image Sender - Simulator")
        self.window.configure(bg="#1a1a1a")
        self.window.state('zoomed')  # Make the window full-screen

        self.style = ttk.Style()
        self.style.configure("TFrame", background="#1a1a1a")
        self.style.configure("TLabel", background="#1a1a1a", foreground="#00ffcc", font=("Orbitron", 12))
        self.style.configure("TButton", background="#00ffcc", foreground="#1a1a1a", font=("Orbitron", 12, "bold"))
        self.style.map("TButton", background=[("active", "#00cc99")])
        self.style.configure("TProgressbar", background="#00ffcc", troughcolor="#333333")

        # Main layout: Sidebar for logs, main area for controls
        self.sidebar = ttk.Frame(self.window, width=400, style="TFrame")
        self.sidebar.grid(row=0, column=0, sticky="ns", padx=10, pady=10)

        self.main_area = ttk.Frame(self.window, style="TFrame")
        self.main_area.grid(row=0, column=1, sticky="nsew", padx=10, pady=10)

        self.window.grid_rowconfigure(0, weight=1)
        self.window.grid_columnconfigure(1, weight=1)

        # Sidebar: Log area
        ttk.Label(self.sidebar, text="Transmission Log", font=("Orbitron", 14, "bold")).pack(pady=10)
        self.log_text = scrolledtext.ScrolledText(self.sidebar, width=50, height=40, bg="#333333", fg="#00ffcc", font=("Orbitron", 10))
        self.log_text.pack(padx=10, pady=10, fill="both", expand=True)

        # Main area: Controls
        ttk.Label(self.main_area, text="Quantum Image Sender", font=("Orbitron", 16, "bold")).grid(row=0, column=0, columnspan=2, pady=20)

        ttk.Button(self.main_area, text="Load Image", command=self.load_image).grid(row=1, column=0, padx=10, pady=10)
        ttk.Button(self.main_area, text="Transmit Image", command=self.transmit_image).grid(row=1, column=1, padx=10, pady=10)
        ttk.Button(self.main_area, text="View NEQR Circuit", command=self.view_neqr_circuit).grid(row=2, column=0, columnspan=2, pady=10)

        # Progress bar
        self.progress = ttk.Progressbar(self.main_area, length=300, mode="indeterminate", style="TProgressbar")
        self.progress.grid(row=3, column=0, columnspan=2, pady=20)

        self.log_queue = queue.Queue()
        self.window.after(100, self.update_log)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png *.jpg *.jpeg")])
        if file_path:
            self.image_bits = image_to_bits(file_path)
            self.log_text.insert(tk.END, f"Loaded image: {len(self.image_bits)} bits\n")
            self.log_text.see(tk.END)

    def transmit_image(self):
        if not hasattr(self, 'image_bits'):
            messagebox.showerror("Error", "Load an image first!")
            return
        self.progress.start()
        threading.Thread(target=self.run_qkd, daemon=True).start()

    def run_qkd(self):
        try:
            transmitted_bits, qtrng_key, encryption_keys, neqr_circuit = transmit_qkd_image(self.image_bits, 24, self.log_queue)
            self.neqr_circuit = neqr_circuit
            self.channel_queue.put((transmitted_bits, qtrng_key, encryption_keys))
            self.status_queue.put("Transmission Complete")
            self.log_queue.put(f"Transmitted bits: {transmitted_bits[:20]}... (Length: {len(transmitted_bits)})")
            self.log_queue.put(f"Quantum Key: {qtrng_key[:20]}... (Length: {len(qtrng_key)})")
            
            qtrng_file, qtrng_fernet_file = encrypt_and_store(qtrng_key, self.usb_path, "qtrng_key")
            confusion_key, permutation = encryption_keys
            confusion_file, confusion_fernet_file = encrypt_and_store(confusion_key, self.usb_path, "confusion_key")
            permutation_file, permutation_fernet_file = encrypt_and_store(permutation, self.usb_path, "permutation")
            
            self.log_queue.put(f"Quantum Key saved to {qtrng_file} with key in {qtrng_fernet_file}")
            self.log_queue.put(f"Confusion key saved to {confusion_file} with key in {confusion_fernet_file}")
            self.log_queue.put(f"Permutation saved to {permutation_file} with key in {permutation_fernet_file}")
            print("QKD transmission finished.")
        except Exception as e:
            self.log_queue.put(f"Error during transmission: {str(e)}")
            print(f"Debug Transmission Error: {str(e)}")
        finally:
            self.progress.stop()

    def view_neqr_circuit(self):
        if not hasattr(self, 'neqr_circuit') or self.neqr_circuit is None:
            self.log_queue.put("Error: NEQR circuit not available. Transmit an image first.")
            return
        NEQRCircuitWindow(self.window, self.neqr_circuit)

    def update_log(self):
        try:
            while not self.log_queue.empty():
                message = self.log_queue.get_nowait()
                self.log_text.insert(tk.END, message + "\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass
        self.window.after(100, self.update_log)

# --- Receiver GUI ---
class ReceiverGUI:
    def __init__(self, parent, channel_queue, status_queue):
        self.channel_queue = channel_queue
        self.status_queue = status_queue
        self.login_window = LoginWindow(parent, self.start_main_gui, "Receiver")
        self.usb_path = None
        self.expected_bits = 2048  # Initialize expected_bits

    def start_main_gui(self, usb_path):
        self.usb_path = usb_path
        self.window = tk.Toplevel(self.login_window.window.master)
        self.window.title("Quantum Image Receiver - Simulator")
        self.window.configure(bg="#1a1a1a")

        self.style = ttk.Style()
        self.style.configure("TFrame", background="#1a1a1a")
        self.style.configure("TLabel", background="#1a1a1a", foreground="#00ffcc", font=("Orbitron", 12))
        self.style.configure("TButton", background="#00ffcc", foreground="#1a1a1a", font=("Orbitron", 12, "bold"))
        self.style.map("TButton", background=[("active", "#00cc99")])
        self.style.configure("TProgressbar", background="#00ffcc", troughcolor="#333333")

        # Main layout: Sidebar for logs, main area for controls
        self.sidebar = ttk.Frame(self.window, width=300, style="TFrame")
        self.sidebar.grid(row=0, column=0, sticky="ns", padx=10, pady=10)

        self.main_area = ttk.Frame(self.window, style="TFrame")
        self.main_area.grid(row=0, column=1, sticky="nsew", padx=10, pady=10)

        # Sidebar: Log area
        ttk.Label(self.sidebar, text="Transmission Log", font=("Orbitron", 14, "bold")).pack(pady=10)
        self.log_text = scrolledtext.ScrolledText(self.sidebar, width=40, height=30, bg="#333333", fg="#00ffcc", font=("Orbitron", 10))
        self.log_text.pack(padx=10, pady=10, fill="both", expand=True)

        # Main area: Controls
        ttk.Label(self.main_area, text="Quantum Image Receiver", font=("Orbitron", 16, "bold")).grid(row=0, column=0, columnspan=2, pady=20)

        ttk.Button(self.main_area, text="Receive Image", command=self.receive_image).grid(row=1, column=0, padx=10, pady=10)
        ttk.Button(self.main_area, text="Save Decrypted Image", command=self.save_image).grid(row=1, column=1, padx=10, pady=10)

        # Progress bar
        self.progress = ttk.Progressbar(self.main_area, length=300, mode="indeterminate", style="TProgressbar")
        self.progress.grid(row=2, column=0, columnspan=2, pady=20)

        self.log_queue = queue.Queue()
        self.window.after(100, self.update_log)

    def receive_image(self):
        self.progress.start()
        threading.Thread(target=self.check_channel, daemon=True).start()

    def check_channel(self):
        try:
            transmitted_bits, qtrng_key, encryption_keys = self.channel_queue.get(timeout=10)
            
            pixels = [transmitted_bits[i:i+8] for i in range(0, len(transmitted_bits), 8)]
            
            decrypted_pixels = decrypt_simulated_image(pixels, encryption_keys, self.log_queue)
            
            xored_bits = []
            for pixel in decrypted_pixels:
                xored_bits.extend(pixel)
            
            self.decrypted_bits = [b ^ k for b, k in zip(xored_bits, qtrng_key)]
            
            self.log_queue.put(f"Received bits: {len(transmitted_bits)} bits")
            self.log_queue.put(f"Decrypted {len(self.decrypted_bits)} bits")
            if len(self.decrypted_bits) != self.expected_bits:
                self.log_queue.put(f"Warning: Expected {self.expected_bits} bits, but received {len(self.decrypted_bits)} bits.")
            self.log_queue.put("Image received successfully!")
            self.status_queue.put("Reception Complete")
        except queue.Empty:
            self.log_queue.put("No image data received yet.")
        except Exception as e:
            self.log_queue.put(f"Error during reception: {str(e)}")
        finally:
            self.progress.stop()

    def save_image(self):
        if not hasattr(self, 'decrypted_bits'):
            messagebox.showerror("Error", "Receive image first!")
            return
        save_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[("PNG files", "*.png"), ("All files", "*.*")]
        )
        if save_path:
            try:
                output_path = bits_to_image(self.decrypted_bits, save_path)
                self.log_queue.put(f"Image saved to {output_path}")
            except Exception as e:
                self.log_queue.put(f"Error saving image: {str(e)}")

    def update_log(self):
        try:
            while not self.log_queue.empty():
                message = self.log_queue.get_nowait()
                self.log_text.insert(tk.END, message + "\n")
                self.log_text.see(tk.END)
        except queue.Empty:
            pass
        self.window.after(100, self.update_log)

# --- Status Monitor GUI ---
class StatusMonitorGUI:
    def __init__(self, parent, status_queue):
        self.window = tk.Toplevel(parent)
        self.window.title("Transmission Status Monitor - Simulator")
        self.window.configure(bg="#1a1a1a")

        self.style = ttk.Style()
        self.style.configure("TFrame", background="#1a1a1a")
        self.style.configure("TLabel", background="#1a1a1a", foreground="#00ffcc", font=("Orbitron", 12))
        self.style.configure("TButton", background="#00ffcc", foreground="#1a1a1a", font=("Orbitron", 12, "bold"))
        self.style.map("TButton", background=[("active", "#00cc99")])

        self.main_frame = ttk.Frame(self.window, style="TFrame")
        self.main_frame.grid(row=0, column=0, padx=20, pady=20)

        ttk.Label(self.main_frame, text="Transmission Status Monitor", font=("Orbitron", 16, "bold")).grid(row=0, column=0, pady=20)

        self.status_label = ttk.Label(self.main_frame, text="Status: Waiting...", font=("Orbitron", 12))
        self.status_label.grid(row=1, column=0, pady=10)

        self.status_queue = status_queue
        self.window.after(100, self.update_status)

    def update_status(self):
        try:
            while not self.status_queue.empty():
                status = self.status_queue.get_nowait()
                self.status_label.config(text=f"Status: {status}")
        except queue.Empty:
            pass
        self.window.after(100, self.update_status)

# --- Main Execution ---
if __name__ == "__main__":
    channel_queue = queue.Queue()
    status_queue = queue.Queue()
    root = tk.Tk()
    root.withdraw()
    sender_app = SenderGUI(root, channel_queue, status_queue)
    receiver_app = ReceiverGUI(root, channel_queue, status_queue)
    status_monitor = StatusMonitorGUI(root, status_queue)
    root.mainloop()

Generated OTP (check console): 858297
Current MAC: 89:cb:c4:10:87:7c
